# Commitment analysis stack compiler
This notebook takes the output of the 4-channel brightfield, gfp-h2b, rfp-h2b, irfp-pcna/JNK/p38 experiments and formats the images into stacks

In [1]:
import napari
print("Napari version no.:", napari.__version__)
import btrack
print('Btrack version', btrack.__version__)
import os, glob, enum
import numpy as np
import tifffile as tiff
from tqdm import tqdm
from skimage.io import imread
from natsort import natsorted
from datetime import datetime
from btrack.utils import tracks_to_napari

Napari version no.: 0.4.11
Btrack version 0.4.2


### Set data folder and number of channels to display

In [3]:
### set root data folder
root_folder = '/home/nathan/data/kraken/commitment/'
expt_IDs = natsorted([ID for ID in os.listdir(root_folder) if 'MK' in ID])
print('Experiment/position list:', [(expt,natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])) for expt in expt_IDs])
### not entirely sure this is the correct use of a class but im still learning
@enum.unique
class channels(enum.Enum):
    #bf = 0 
    gfp = 1
    rfp = 2
    #irfp = 3
    cp_masks = 4

Experiment/position list: [('MK0000', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15', 'Pos16', 'Pos17', 'Pos18', 'Pos19', 'Pos20', 'Pos21', 'Pos22', 'Pos23']), ('MK0000v1', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15', 'Pos16', 'Pos17', 'Pos18', 'Pos19', 'Pos20', 'Pos21', 'Pos22', 'Pos23']), ('MK0001', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15']), ('MK0002', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15']), ('MK0003', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15'])]


Select one position if necessary

In [4]:
expt = 'MK0002'
pos = 'Pos12'#'Pos15'

data_folder = os.path.join(root_folder, '{}/{}'.format(expt, pos)) #'/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15'#'/media/nathan/Transcend/Nathan/data/pcna/test_210422/Pos7'
#data_folder = os.path.join(root_folder, '{}'.format(pos)) #'/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15'#'/media/nathan/Transcend/Nathan/data/pcna/test_210422/Pos7'

### Compile folders of single frame images into stacks if necessary

In [19]:
## move raw files into own folder inside 
if '{}_raw'.format(pos) not in os.listdir(data_folder):
    os.mkdir(os.path.join(data_folder,'{}_raw'.format(pos)))
    print(output_dir, 'directory created')
    files = glob.glob(os.path.join(data_folder, 'img_channel00*.tif'))
    for file in tqdm(files):
        os.rename(file, file.replace('img', f'{pos}_raw/img'))

In [5]:
data_folder

'/home/nathan/data/kraken/commitment/MK0002/Pos12'

In [6]:
## compile stacks
output_dir = os.path.join(data_folder,'{}_stacks'.format(pos))
if '{}_stacks'.format(pos) not in os.listdir(data_folder):
    os.mkdir(os.path.join(data_folder,'{}_stacks'.format(pos)))
    print(output_dir, 'directory created')
    for channel in channels:
        print(channel.name, 'compiling')
        files = glob.glob(os.path.join(data_folder, 
                                       '{}_raw'.format(pos),
                                       'img_channel00{}*.tif'.format(channel.value)))
        stack = []
        for file in tqdm(natsorted(files)):
            tif = np.array(tiff.imread(file),dtype=np.uint8)
            tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
            tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
            stack.append(tif)
        stack = np.stack(stack,axis=0)
        with tiff.TiffWriter(os.path.join(output_dir,'{}.tif'.format(channel.name)),imagej=True) as output_file:
            output_file.save(stack)
        print(channel.name, 'saved out')
else:
    print('looks like stacks already exist bro')

/home/nathan/data/kraken/pcna/MK0002/Pos5/Pos5_stacks directory created
bf compiling


100%|██████████| 1200/1200 [00:16<00:00, 71.09it/s]


bf saved out
gfp compiling


100%|██████████| 1200/1200 [00:55<00:00, 21.47it/s]


gfp saved out
rfp compiling


100%|██████████| 1200/1200 [00:58<00:00, 20.65it/s]


rfp saved out
irfp compiling


100%|██████████| 1200/1200 [00:51<00:00, 23.12it/s]


irfp saved out


In [7]:
## compile mask stacks
output_dir = os.path.join(data_folder,'{}_stacks'.format(pos))

for channel in channels:
        print(channel.name, 'compiling')
        files = glob.glob(os.path.join(data_folder, 
                                       '{}_cp_masks'.format(pos),
                                       'img_channel00{}*.png'.format(channel.value)))
        stack = []
        for file in tqdm(natsorted(files)):
            tif = np.array(imread(file),dtype=np.uint8)
            stack.append(tif)
        stack = np.stack(stack,axis=0)
        with tiff.TiffWriter(os.path.join(output_dir,'{}_cp_masks.tif'.format(channel.name)),imagej=True) as output_file:
            output_file.save(stack)
        print(channel.name, 'saved out')

gfp compiling


100%|██████████| 1200/1200 [00:39<00:00, 30.20it/s]


gfp saved out
rfp compiling


100%|██████████| 1200/1200 [00:51<00:00, 23.41it/s]


rfp saved out
cp_masks compiling


0it [00:00, ?it/s]


ValueError: need at least one array to stack

#### Compile multiple experiments and positions from single frame images into stacks

In [38]:
for expt in 'MK0000':
    ### get all positions in that experiment
    pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])
    for pos in tqdm(pos_list):
        print('Starting', expt, pos)
        data_folder = os.path.join(root_folder, f'{expt}/{pos}')
        ### check if stacks already made
        if f'{pos}_stacks' not in os.listdir(data_folder):
            ### make output dir for stacks and input dir for raw files and cp mask directory
            os.mkdir(os.path.join(data_folder,f'{pos}_stacks'))
            os.mkdir(os.path.join(data_folder,f'{pos}_raw'))
            os.mkdir(os.path.join(data_folder, f'{pos}_cp_masks'))
            print('Output, input and cellpose mask directories created')
            ### move raw images into directory
            files = glob.glob(os.path.join(data_folder, 'img_channel00*'))
            for file in files:
                ### if cp_mask then move into cp_mask dir
                if 'cp_mask' in file:
                    os.rename(file, file.replace('img', f'{pos}_cp_masks/img'))
                ### if standard file then move there
                else:
                    os.rename(file, file.replace('img', f'{pos}_raw/img'))
            print(expt, pos, 'files moved into separate directories')
            for channel in channels:
                print(channel.name, 'compiling')
                if channel.name == 'cp_masks':
                    files = glob.glob(os.path.join(data_folder, 
                               f'{pos}_cp_masks',
                               f'img_channel00*.png'))
                else:
                    files = glob.glob(os.path.join(data_folder, 
                                               f'{pos}_raw',
                                               f'img_channel00{channel.value}*.tif'))
                stack = []
                if files == []:
                    continue
                for file in natsorted(files):
                    if channel.name == 'cp_masks':
                        tif = np.array(imread(file), dtype=np.uint8)
                    else:
                        tif = np.array(tiff.imread(file),dtype=np.uint8)
                    ### unnecessary bc images are not aligned
                    #tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
                    #tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
                    stack.append(tif)
                stack = np.stack(stack,axis=0)
                with tiff.TiffWriter(os.path.join(data_folder,f'{pos}_stacks',f'{channel.name}.tif'),imagej=True) as output_file:
                    output_file.save(stack)
                print(channel.name, 'saved out')
        else:
            print('looks like stacks already exist bro')
        
     

100%|██████████| 24/24 [00:00<00:00, 1451.32it/s]


Starting MK0000 Pos0
looks like stacks already exist bro
Starting MK0000 Pos1
looks like stacks already exist bro
Starting MK0000 Pos2
looks like stacks already exist bro
Starting MK0000 Pos3
looks like stacks already exist bro
Starting MK0000 Pos4
looks like stacks already exist bro
Starting MK0000 Pos5
looks like stacks already exist bro
Starting MK0000 Pos6
looks like stacks already exist bro
Starting MK0000 Pos7
looks like stacks already exist bro
Starting MK0000 Pos8
looks like stacks already exist bro
Starting MK0000 Pos9
looks like stacks already exist bro
Starting MK0000 Pos10
looks like stacks already exist bro
Starting MK0000 Pos11
looks like stacks already exist bro
Starting MK0000 Pos12
looks like stacks already exist bro
Starting MK0000 Pos13
looks like stacks already exist bro
Starting MK0000 Pos14
looks like stacks already exist bro
Starting MK0000 Pos15
looks like stacks already exist bro
Starting MK0000 Pos16
looks like stacks already exist bro
Starting MK0000 Pos17
lo


  0%|          | 0/16 [00:00<?, ?it/s]

Starting MK0001 Pos0



  6%|▋         | 1/16 [00:00<00:02,  5.16it/s]

...

irfp saved out
cp_masks compiling
Starting MK0003 Pos14
Output, input and cellpose mask directories created
MK0003 Pos14 files moved into separate directories
bf compiling
bf saved out
gfp compiling
gfp saved out
rfp compiling
rfp saved out
irfp compiling



 94%|█████████▍| 15/16 [1:00:14<04:01, 241.53s/it]

irfp saved out
cp_masks compiling
Starting MK0003 Pos15
Output, input and cellpose mask directories created
MK0003 Pos15 files moved into separate directories
bf compiling
bf saved out
gfp compiling
gfp saved out
rfp compiling
rfp saved out
irfp compiling



100%|██████████| 4/4 [2:04:36<00:00, 1869.05s/it]

irfp saved out
cp_masks compiling


# sort out MK0000

need naming convention to be bf, gfp, rfp, irfp, cp_masks

also need to make stack of cellpose masks

In [33]:
root_dir = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000'
pos_list = natsorted([pos for pos in os.listdir(root_dir) if 'Pos' in pos])
print(pos_list)

['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15', 'Pos16', 'Pos17', 'Pos18', 'Pos19', 'Pos20', 'Pos21', 'Pos22', 'Pos23']


In [60]:
@enum.unique
class channels(enum.Enum):
    bf = 0 
    gfp = 1
    rfp = 2
    irfp = 3
    cp_masks = 4

In [62]:
root_folder = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/'
for expt in ['MK0000']:
    ### get all positions in that experiment
    pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])
    for pos in tqdm(pos_list):
        print('Starting', expt, pos)
        data_folder = os.path.join(root_folder, f'{expt}/{pos}')
        
        for channel in channels:
            print(channel.name, 'compiling')
            if channel.name == 'cp_masks':
                files = glob.glob(os.path.join(data_folder, 
                           f'{pos}_cp_masks',
                           f'img_channel001*.png'))
            else:
                files = glob.glob(os.path.join(data_folder, 
                                           f'{pos}_raw',
                                           f'img_channel00{channel.value}*.tif'))
            stack = []
            if files == []:
                continue
            for file in natsorted(files):
                if channel.name == 'cp_masks':
                    tif = np.array(imread(file), dtype=np.uint8)
                else:
                    tif = np.array(tiff.imread(file),dtype=np.uint8)
                ### unnecessary bc images are not aligned
                #tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
                #tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
                stack.append(tif)
            stack = np.stack(stack,axis=0)
            with tiff.TiffWriter(os.path.join(data_folder,f'{pos}_stacks',f'{channel.name}.tif'),imagej=True) as output_file:
                output_file.save(stack)
            print(channel.name, 'saved out')
        

  0%|          | 0/24 [00:00<?, ?it/s]

Starting MK0000 Pos0
bf compiling
bf saved out
gfp compiling
gfp saved out
rfp compiling
rfp saved out
irfp compiling
irfp saved out
cp_masks compiling


  4%|▍         | 1/24 [10:34<4:03:22, 634.89s/it]

cp_masks saved out
Starting MK0000 Pos1
bf compiling
bf saved out
gfp compiling
gfp saved out
rfp compiling
rfp saved out
irfp compiling
irfp saved out
cp_masks compiling


  8%|▊         | 2/24 [19:01<3:25:13, 559.71s/it]

...

cp_masks saved out
Starting MK0000 Pos22
bf compiling
bf saved out
gfp compiling
gfp saved out
rfp compiling
rfp saved out
irfp compiling
irfp saved out
cp_masks compiling


 96%|█████████▌| 23/24 [3:38:09<10:04, 604.76s/it]

cp_masks saved out
Starting MK0000 Pos23
bf compiling
bf saved out
gfp compiling
gfp saved out
rfp compiling
rfp saved out
irfp compiling


100%|██████████| 24/24 [3:46:32<00:00, 566.37s/it]

irfp saved out
cp_masks compiling


In [66]:
for pos in tqdm(pos_list):
    print('Starting', expt, pos)
    data_folder = os.path.join(root_folder, f'{expt}/{pos}/{pos}_stacks')
    files = glob.glob(os.path.join(data_folder, 'channel*.tif'))
    for file in files:
        print(file, 'removing')
        os.remove(file)

  0%|          | 0/24 [00:00<?, ?it/s]

Starting MK0000 Pos0
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos0/Pos0_stacks/channel002.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos0/Pos0_stacks/channel000.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos0/Pos0_stacks/channel001.tif removing


  4%|▍         | 1/24 [00:00<00:16,  1.36it/s]

/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos0/Pos0_stacks/channel003.tif removing
Starting MK0000 Pos1
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos1/Pos1_stacks/channel002.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos1/Pos1_stacks/channel000.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos1/Pos1_stacks/channel001.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos1/Pos1_stacks/channel003.tif removing


  8%|▊         | 2/24 [00:01<00:13,  1.62it/s]

...

/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos21/Pos21_stacks/channel001.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos21/Pos21_stacks/channel003.tif removing
Starting MK0000 Pos22
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos22/Pos22_stacks/channel002.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos22/Pos22_stacks/channel000.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos22/Pos22_stacks/channel001.tif removing


 96%|█████████▌| 23/24 [00:11<00:00,  1.93it/s]

/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos22/Pos22_stacks/channel003.tif removing
Starting MK0000 Pos23
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos23/Pos23_stacks/channel002.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos23/Pos23_stacks/channel000.tif removing
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos23/Pos23_stacks/channel001.tif removing


100%|██████████| 24/24 [00:12<00:00,  1.95it/s]

/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos23/Pos23_stacks/channel003.tif removing


In [51]:
for pos in (pos_list):
    raw_dir = os.path.join(root_dir, pos, f'{pos}_raw')
    mask_dir = os.path.join(root_dir, pos, f'{pos}_cp_masks')
    cp_masks = natsorted(glob.glob(os.path.join(mask_dir, '*.png')))
    for i in cp_masks:
        #print(pos, i.split('/')[-1])
        if 'channel001' not in i:
            print(i.split('/')[-1])

img_channel002_position000_time000000000_z000_cp_masks.png
img_channel002_position000_time000000001_z000_cp_masks.png
img_channel002_position000_time000000002_z000_cp_masks.png
img_channel002_position000_time000000003_z000_cp_masks.png
img_channel002_position000_time000000004_z000_cp_masks.png
img_channel002_position000_time000000005_z000_cp_masks.png
img_channel002_position000_time000000006_z000_cp_masks.png
img_channel002_position000_time000000007_z000_cp_masks.png
img_channel002_position000_time000000008_z000_cp_masks.png
img_channel002_position000_time000000009_z000_cp_masks.png
img_channel002_position000_time000000010_z000_cp_masks.png
img_channel002_position000_time000000011_z000_cp_masks.png
img_channel002_position000_time000000012_z000_cp_masks.png
img_channel002_position000_time000000013_z000_cp_masks.png
img_channel002_position000_time000000014_z000_cp_masks.png
img_channel002_position000_time000000015_z000_cp_masks.png
img_channel002_position000_time000000016_z000_cp_masks.p

img_channel003_position001_time000000000_z000_cp_masks.png
img_channel003_position001_time000000001_z000_cp_masks.png
img_channel003_position001_time000000002_z000_cp_masks.png
img_channel003_position001_time000000003_z000_cp_masks.png
img_channel003_position001_time000000004_z000_cp_masks.png
img_channel003_position001_time000000005_z000_cp_masks.png
img_channel003_position001_time000000006_z000_cp_masks.png
img_channel003_position001_time000000007_z000_cp_masks.png
img_channel003_position001_time000000008_z000_cp_masks.png
img_channel003_position001_time000000009_z000_cp_masks.png
img_channel003_position001_time000000010_z000_cp_masks.png
img_channel003_position001_time000000011_z000_cp_masks.png
img_channel003_position001_time000000012_z000_cp_masks.png
img_channel003_position001_time000000013_z000_cp_masks.png
img_channel003_position001_time000000014_z000_cp_masks.png
img_channel003_position001_time000000015_z000_cp_masks.png
img_channel003_position001_time000000016_z000_cp_masks.p

img_channel003_position002_time000000000_z000_cp_masks.png
img_channel003_position002_time000000001_z000_cp_masks.png
img_channel003_position002_time000000002_z000_cp_masks.png
img_channel003_position002_time000000003_z000_cp_masks.png
img_channel003_position002_time000000004_z000_cp_masks.png
img_channel003_position002_time000000005_z000_cp_masks.png
img_channel003_position002_time000000006_z000_cp_masks.png
img_channel003_position002_time000000007_z000_cp_masks.png
img_channel003_position002_time000000008_z000_cp_masks.png
img_channel003_position002_time000000009_z000_cp_masks.png
img_channel003_position002_time000000010_z000_cp_masks.png
img_channel003_position002_time000000011_z000_cp_masks.png
img_channel003_position002_time000000012_z000_cp_masks.png
img_channel003_position002_time000000013_z000_cp_masks.png
img_channel003_position002_time000000014_z000_cp_masks.png
img_channel003_position002_time000000015_z000_cp_masks.png
img_channel003_position002_time000000016_z000_cp_masks.p

img_channel002_position005_time000000000_z000_cp_masks.png
img_channel002_position005_time000000001_z000_cp_masks.png
img_channel002_position005_time000000002_z000_cp_masks.png
img_channel002_position005_time000000003_z000_cp_masks.png
img_channel002_position005_time000000004_z000_cp_masks.png
img_channel002_position005_time000000005_z000_cp_masks.png
img_channel002_position005_time000000006_z000_cp_masks.png
img_channel002_position005_time000000007_z000_cp_masks.png
img_channel002_position005_time000000008_z000_cp_masks.png
img_channel002_position005_time000000009_z000_cp_masks.png
img_channel002_position005_time000000010_z000_cp_masks.png
img_channel002_position005_time000000011_z000_cp_masks.png
img_channel002_position005_time000000012_z000_cp_masks.png
img_channel002_position005_time000000013_z000_cp_masks.png
img_channel002_position005_time000000014_z000_cp_masks.png
img_channel002_position005_time000000015_z000_cp_masks.png
img_channel002_position005_time000000016_z000_cp_masks.p

img_channel002_position006_time000000000_z000_cp_masks.png
img_channel002_position006_time000000001_z000_cp_masks.png
img_channel002_position006_time000000002_z000_cp_masks.png
img_channel002_position006_time000000003_z000_cp_masks.png
img_channel002_position006_time000000004_z000_cp_masks.png
img_channel002_position006_time000000005_z000_cp_masks.png
img_channel002_position006_time000000006_z000_cp_masks.png
img_channel002_position006_time000000007_z000_cp_masks.png
img_channel002_position006_time000000008_z000_cp_masks.png
img_channel002_position006_time000000009_z000_cp_masks.png
img_channel002_position006_time000000010_z000_cp_masks.png
img_channel002_position006_time000000011_z000_cp_masks.png
img_channel002_position006_time000000012_z000_cp_masks.png
img_channel002_position006_time000000013_z000_cp_masks.png
img_channel002_position006_time000000014_z000_cp_masks.png
img_channel002_position006_time000000015_z000_cp_masks.png
img_channel002_position006_time000000016_z000_cp_masks.p

...

img_channel002_position016_time000000000_z000_cp_masks.png
img_channel002_position016_time000000001_z000_cp_masks.png
img_channel002_position016_time000000002_z000_cp_masks.png
img_channel002_position016_time000000003_z000_cp_masks.png
img_channel002_position016_time000000004_z000_cp_masks.png
img_channel002_position016_time000000005_z000_cp_masks.png
img_channel002_position016_time000000006_z000_cp_masks.png
img_channel002_position016_time000000007_z000_cp_masks.png
img_channel002_position016_time000000008_z000_cp_masks.png
img_channel002_position016_time000000009_z000_cp_masks.png
img_channel002_position016_time000000010_z000_cp_masks.png
img_channel002_position016_time000000011_z000_cp_masks.png
img_channel002_position016_time000000012_z000_cp_masks.png
img_channel002_position016_time000000013_z000_cp_masks.png
img_channel002_position016_time000000014_z000_cp_masks.png
img_channel002_position016_time000000015_z000_cp_masks.png
img_channel002_position016_time000000016_z000_cp_masks.p

img_channel002_position017_time000000000_z000_cp_masks.png
img_channel002_position017_time000000001_z000_cp_masks.png
img_channel002_position017_time000000002_z000_cp_masks.png
img_channel002_position017_time000000003_z000_cp_masks.png
img_channel002_position017_time000000004_z000_cp_masks.png
img_channel002_position017_time000000005_z000_cp_masks.png
img_channel002_position017_time000000006_z000_cp_masks.png
img_channel002_position017_time000000007_z000_cp_masks.png
img_channel002_position017_time000000008_z000_cp_masks.png
img_channel002_position017_time000000009_z000_cp_masks.png
img_channel002_position017_time000000010_z000_cp_masks.png
img_channel002_position017_time000000011_z000_cp_masks.png
img_channel002_position017_time000000012_z000_cp_masks.png
img_channel002_position017_time000000013_z000_cp_masks.png
img_channel002_position017_time000000014_z000_cp_masks.png
img_channel002_position017_time000000015_z000_cp_masks.png
img_channel002_position017_time000000016_z000_cp_masks.p

img_channel002_position020_time000000000_z000_cp_masks.png
img_channel002_position020_time000000001_z000_cp_masks.png
img_channel002_position020_time000000002_z000_cp_masks.png
img_channel002_position020_time000000003_z000_cp_masks.png
img_channel002_position020_time000000004_z000_cp_masks.png
img_channel002_position020_time000000005_z000_cp_masks.png
img_channel002_position020_time000000006_z000_cp_masks.png
img_channel002_position020_time000000007_z000_cp_masks.png
img_channel002_position020_time000000008_z000_cp_masks.png
img_channel002_position020_time000000009_z000_cp_masks.png
img_channel002_position020_time000000010_z000_cp_masks.png
img_channel002_position020_time000000011_z000_cp_masks.png
img_channel002_position020_time000000012_z000_cp_masks.png
img_channel002_position020_time000000013_z000_cp_masks.png
img_channel002_position020_time000000014_z000_cp_masks.png
img_channel002_position020_time000000015_z000_cp_masks.png
img_channel002_position020_time000000016_z000_cp_masks.p

img_channel002_position021_time000000000_z000_cp_masks.png
img_channel002_position021_time000000001_z000_cp_masks.png
img_channel002_position021_time000000002_z000_cp_masks.png
img_channel002_position021_time000000003_z000_cp_masks.png
img_channel002_position021_time000000004_z000_cp_masks.png
img_channel002_position021_time000000005_z000_cp_masks.png
img_channel002_position021_time000000006_z000_cp_masks.png
img_channel002_position021_time000000007_z000_cp_masks.png
img_channel002_position021_time000000008_z000_cp_masks.png
img_channel002_position021_time000000009_z000_cp_masks.png
img_channel002_position021_time000000010_z000_cp_masks.png
img_channel002_position021_time000000011_z000_cp_masks.png
img_channel002_position021_time000000012_z000_cp_masks.png
img_channel002_position021_time000000013_z000_cp_masks.png
img_channel002_position021_time000000014_z000_cp_masks.png
img_channel002_position021_time000000015_z000_cp_masks.png
img_channel002_position021_time000000016_z000_cp_masks.p

img_channel002_position022_time000000000_z000_cp_masks.png
img_channel002_position022_time000000001_z000_cp_masks.png
img_channel002_position022_time000000002_z000_cp_masks.png
img_channel002_position022_time000000003_z000_cp_masks.png
img_channel002_position022_time000000004_z000_cp_masks.png
img_channel002_position022_time000000005_z000_cp_masks.png
img_channel002_position022_time000000006_z000_cp_masks.png
img_channel002_position022_time000000007_z000_cp_masks.png
img_channel002_position022_time000000008_z000_cp_masks.png
img_channel002_position022_time000000009_z000_cp_masks.png
img_channel002_position022_time000000010_z000_cp_masks.png
img_channel002_position022_time000000011_z000_cp_masks.png
img_channel002_position022_time000000012_z000_cp_masks.png
img_channel002_position022_time000000013_z000_cp_masks.png
img_channel002_position022_time000000014_z000_cp_masks.png
img_channel002_position022_time000000015_z000_cp_masks.png
img_channel002_position022_time000000016_z000_cp_masks.p

In [59]:
for pos in (pos_list):
    raw_dir = os.path.join(root_dir, pos, f'{pos}_raw')
    print(len(os.listdir(raw_dir)))

4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056
4056


In [53]:
4056/4

1014.0

In [21]:
root_dir = '/home/nathan/data/kraken/commitment/MK0000'
pos_list = natsorted([pos for pos in os.listdir(root_dir) if 'Pos' in pos])
print(pos_list)

['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15', 'Pos16', 'Pos17', 'Pos18', 'Pos19', 'Pos20', 'Pos21', 'Pos22', 'Pos23']


In [31]:
5067/5

1013.4

In [32]:
5070/5

1014.0

In [29]:
for pos in pos_list:
    ### rename stacks
    #stack_dir = os.path.join(root_dir, pos, f'{pos}_stacks')
    #files = natsorted(glob.glob(os.path.join(stack_dir, '*.tif')))
#     os.rename(files[0], files[0].replace('channel000', 'bf')) 
#     os.rename(files[1], files[1].replace('channel001', 'gfp')) 
#     os.rename(files[2], files[2].replace('channel002', 'rfp')) 
#     os.rename(files[3], files[3].replace('channel003', 'irfp')) 
    ### compile cp masks into cp mask stack
    
    for channel in ['1', '2']:
        files = natsorted(glob.glob(os.path.join(root_dir, pos, 
                                       f'{pos}_cp_masks',
                                       f'img_channel00{channel}*.png')))
        print(pos, channel, len(files))
        raw_files = 
#         stack = []
#         for file in tqdm(natsorted(files)):
#             tif = np.array(imread(file),dtype=np.uint8)
#             stack.append(tif)
#         stack = np.stack(stack,axis=0)
#         with tiff.TiffWriter(os.path.join(stack_dir,'cp_masks.tif'),imagej=True) as output_file:
#             output_file.save(stack)
#         print( 'saved out')

Pos0 1 1014
Pos0 2 451
Pos1 1 0
Pos1 2 0
Pos2 1 0
Pos2 2 0
Pos3 1 0
Pos3 2 0
Pos4 1 0
Pos4 2 0
Pos5 1 1014
Pos5 2 1014
Pos6 1 1014
Pos6 2 1014
Pos7 1 1014
Pos7 2 782
Pos8 1 0
Pos8 2 0
Pos9 1 0
Pos9 2 0
Pos10 1 1014
Pos10 2 1014
Pos11 1 1014
Pos11 2 1014
Pos12 1 1014
Pos12 2 752
Pos13 1 0
Pos13 2 0
Pos14 1 0
Pos14 2 0
Pos15 1 1013
Pos15 2 1014
Pos16 1 1014
Pos16 2 1014
Pos17 1 1014
Pos17 2 670
Pos18 1 0
Pos18 2 0
Pos19 1 0
Pos19 2 0
Pos20 1 1014
Pos20 2 1014
Pos21 1 1014
Pos21 2 1014
Pos22 1 1014
Pos22 2 800
Pos23 1 0
Pos23 2 0


In [17]:
## compile mask stacks
output_dir = os.path.join('/home/nathan/data/kraken/commitment/MK0000/Pos0/Pos0_stacks')
pos = 'Pos0'
files = glob.glob(os.path.join('/home/nathan/data/kraken/commitment/MK0001/Pos0/', 
                               '{}_cp_masks'.format(pos),
                               'img_channel00{}*.png'.format('1')))
stack = []
for file in tqdm(natsorted(files)):
    tif = np.array(imread(file),dtype=np.uint8)
    stack.append(tif)
stack = np.stack(stack,axis=0)
with tiff.TiffWriter(os.path.join(output_dir,'cp_masks.tif'),imagej=True) as output_file:
    output_file.save(stack)
print( 'saved out')

100%|██████████| 1200/1200 [00:35<00:00, 33.55it/s]


saved out
